In [1]:
import os
import re
from glob import glob
import time
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
from torchmetrics.classification import MulticlassMatthewsCorrCoef
import open3d as o3
# from open3d import JVisualizer # For Colab Visualization
# from open3d.web_visualizer import draw # for non Colab

import matplotlib.pyplot as plt
%matplotlib inline

## Set paths

In [2]:
# dataset
# ROOT = r'C:\Users\itber\Documents\datasets\S3DIS\Stanford3dDataset_v1.2_Reduced_Parti\tioned_Aligned_Version_1m'

root = "E:\\Nick\\Cole Trapnell's Lab Dropbox\\Nick Lammers\\Nick\\pecfin_dynamics\\fin_morphodynamics\\"
data_root = os.path.join(root, "built_data\\point_clouds\\")

# feature selection hyperparameters
NUM_TRAIN_POINTS = 4096 # train/valid points
NUM_TEST_POINTS = 4096
BATCH_SIZE = 16

## Get dataloader

In [3]:
from torch.utils.data import DataLoader
from fin_morphodynamics.src.functions.data_utilities import PointData

# get datasets
# point_data = PointData(ROOT, npoints=NUM_TRAIN_POINTS, r_prob=0.25)
# valid_data = PointData(ROOT, npoints=NUM_TRAIN_POINTS, r_prob=0.)
point_data = PointData(data_root, split='test', npoints=NUM_TEST_POINTS)

# get dataloaders
dataloader = DataLoader(point_data, batch_size=BATCH_SIZE, shuffle=True)
# valid_dataloader = DataLoader(s3dis_valid, batch_size=BATCH_SIZE, shuffle=True)
# test_dataloader = DataLoader(s3dis_test, batch_size=BATCH_SIZE, shuffle=False)

## Load pretrained model

In [4]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

In [5]:
from fin_morphodynamics.src.point_net.point_net import PointNetBackbone, PointNetSegHead

code_root = "C:\\Users\\nlammers\\Projects\\pecfin_dynamics\\fin_morphodynamics\\"
model_path = os.path.join(code_root, "src\point_net\\trained_models\\seg_focal\seg_model_60.pth")

# point_mdl = torch.load(model_path)
# point_mdl.eval()
# point_mdl_bck = PointNetBackbone(local_feat=True, num_points=4096).eval()
# point_mdl_bck.load_state_dict(torch.load(model_path))

# get intitial model architecture
# model = PointNetSegHead(num_points=NUM_TRAIN_POINTS, m=NUM_CLASSES)
NUM_CLASSES = 14

model = PointNetSegHead(num_points=NUM_TEST_POINTS, m=NUM_CLASSES).to(DEVICE)
model.load_state_dict(torch.load(model_path))
model.eval();

## Apply to some sample points and visualize

In [6]:
points, targets = next(iter(dataloader))
points = torch.transpose(points, 1, 2)

In [7]:
points = points.to(DEVICE)
pointfeat = model.backbone
out, _, _ = pointfeat(points)

In [11]:
import umap.umap_ as umap
from sklearn.preprocessing import StandardScaler

# initialize reducer
n_components = 2
reducer = umap.UMAP(n_components=n_components)

        
features = np.squeeze(out[0, :, :]).detach().cpu().T

scaled_features = StandardScaler().fit_transform(features)
embedding = reducer.fit_transform(scaled_features)


In [14]:
import plotly.express as px
import plotly.offline as pyo
pyo.init_notebook_mode()

points_cpu = np.squeeze(points[0, :, :].detach().cpu()).T
fig = px.scatter_3d(x=points_cpu[:, 0], y=points_cpu[:, 1], z=-points_cpu[:, 2], color=embedding[:, 1])
fig.show()

In [10]:
features.shape